# Introduction to LangChain v0.2.0 and LCEL: LangChain Powered RAG

In the following notebook we're going to focus on learning how to navigate and build useful applications using LangChain, specifically LCEL, and how to integrate different APIs together into a coherent RAG application!

In the notebook, you'll complete the following Tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Initialize a Simple Chain using LCEL
  4. Implement Naive RAG using LCEL
  
Let's get started!



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

One of the [key features](https://blog.langchain.dev/langchain-v02-leap-to-stability/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages and added stability.

Instead of one all encompassing Python package - LangChain has a `core` package and a number of additional supplementary packages.

We'll start by grabbing all of our LangChain related packages!

In [43]:
%pip install -qU langchain langchain-core langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


Now we can get our Qdrant dependencies!

In [44]:
%pip install -qU qdrant-client

Note: you may need to restart the kernel to use updated packages.


Let's finally get `tiktoken` and `pymupdf` so we can leverage them later on!

In [45]:
%pip install -qU tiktoken pymupdf

Note: you may need to restart the kernel to use updated packages.


## Task 2: Set Environment Variables

We'll be leveraging OpenAI's suite of APIs - so we'll set our `OPENAI_API_KEY` `env` variable here!

In [46]:
import os
import getpass
from dotenv import load_dotenv 
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Task 3: Initialize a Simple Chain using LCEL

The first thing we'll do is familiarize ourselves with LCEL and the specific ins and outs of how we can use it!

### LLM Orchestration Tool (LangChain)

Let's dive right into [LangChain](https://www.langchain.com/)!

The first thing we want to do is create an object that lets us access OpenAI's `gpt-4o` model.

In [47]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

####❓ Question #1:

What other models could we use, and how would the above code change?
```we could use GPT-4o mini", which is more affordable and useful in lightweight tasks openai_chat_model = ChatOpenAI(model="GPT-4o mini")```

> HINT: Check out [this page](https://platform.openai.com/docs/models) to find the answer!

### Prompt Template

Now, we'll set up a prompt template - more specifically a `ChatPromptTemplate`. This will let us build a prompt we can modify when we call our LLM!

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are a legendary and mythical Wizard. You speak in riddles and make obscure and pun-filled references to exotic cheeses."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain

Now we can set up our first chain!

A chain is simply two components that feed directly into eachother in a sequential fashion!

You'll notice that we're using the pipe operator `|` to connect our `chat_prompt` to our `llm`.

This is a simplified method of creating chains and it leverages the LangChain Expression Language, or LCEL.

You can read more about it [here](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel), but there a few features we should be aware of out of the box (taken directly from LangChain's documentation linked above):

- **Async, Batch, and Streaming Support** Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

- **Fallbacks** The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

- **Parallelism** Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

In the following code cell we have two components:

- `chat_prompt`, which is a formattable `ChatPromptTemplate` that contains a system message and a human message.
- `openai_chat_model`, which is a LangChain Runnable wrapped OpenAI client.

We'd like to be able to pass our own `content` (as found in our `human_template`) and then have the resulting message pair sent to our model and responded to!

In [15]:
chain = chat_prompt | openai_chat_model

Notice the pattern here:

We invoke our chain with the `dict` `{"content" : "Hello world!"}`.

It enters our chain:

`{"content" : "Hello world!"}` -> `invoke()` -> `chat_prompt`

Our `chat_prompt` returns a `PromptValue`, which is the formatted prompt. We then "pipe" the output of our `chat_prompt` into our `llm`.

`PromptValue` -> `|` -> `llm`

Our `llm` then takes the list of messages and provides an output which is return as a `str`!







In [16]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, greetings, seeker of knowledge! Like the rare and elusive Casu Marzu, you have stumbled upon a hidden treasure. What mysteries do you wish to unravel?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 38, 'total_tokens': 72}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None} id='run-61874c4e-f340-4324-912f-fc9c38ee1738-0' usage_metadata={'input_tokens': 38, 'output_tokens': 34, 'total_tokens': 72}


Let's try it out with a different prompt!

In [17]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content="Ah, seeker of serpentine scripts and digital delights, lend me thine ears, for I shall bestow upon thee a riddle wrapped in wisdom, seasoned with the essence of the mighty Roquefort!\n\nIn the realm of Python, where the code doth twist and turn,\nSeek ye answers and knowledge, for which thy heart doth yearn.\nFirst, grasp the basics, as one might savor a mild Brie,\nFor without a firm foundation, lost in syntax ye shall be.\n\nNext, practice thou must, as if maturing a fine Gouda,\nFor repetition and patience maketh thy skills cruda.\nEngage with communities, as diverse as a cheese board,\nFor in sharing and learning, wisdom is stored.\n\nWrite code both simple and complex, like a tale of Cheddar,\nFrom the sharp and tangy, to the smooth and better.\nSolve puzzles and challenges, akin to a Stilton's mold,\nFor in each problem conquered, thy skill shall unfold.\n\nRefactor thy code, like aging a Parmigiano,\nImproving and refining, until it shines like a piano.\nRead 

Notice how we specifically referenced our `content` format option!

Now that we have the basics set up - let's see what we mean by "Retrieval Augmented" Generation.

## Naive RAG - Manually adding context through the Prompt Template

Let's look at how our model performs at a simple task - defining what LangChain is!

We'll redo some of our previous work to change the `system_template` to be less...verbose.

In [18]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content='LangChain is a framework designed to facilitate the development of applications that are powered by large language models (LLMs). It offers modular components and abstractions that simplify the integration of LLMs into various applications, enabling developers to build sophisticated and scalable language-based solutions.\n\nKey features of LangChain include:\n\n1. **LLM Wrappers**: These are interfaces that allow easy interaction with different language models. They provide a standardized way to utilize various LLMs.\n\n2. **Prompt Management**: LangChain includes tools for creating, managing, and optimizing prompts, which are essential for getting the desired responses from language models.\n\n3. **Memory Management**: This feature helps in maintaining the state and context of interactions, which is crucial for applications that require continuity in conversations or tasks.\n\n4. **Data Augmented Generation**: This component integrates external data sources to enhance the res

Well, that's not very good - is it!

The issue at play here is that our model was not trained on the idea of "LangChain", and so it's left with nothing but a guess - definitely not what we want the answer to be!

Let's ask another simple LangChain question!

In [19]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

content='LangChain Expression Language (LECL) is a domain-specific language designed for use within the LangChain framework, which is a library for building applications with large language models (LLMs). LECL allows users to define and manipulate the behavior of language models in a more structured and programmable way. It offers a syntax and set of operations tailored to the needs of working with LLMs, enabling more precise control over tasks such as text generation, data extraction, and workflow automation.\n\nKey features of LECL may include:\n\n1. **Structured Syntax**: Similar to other programming languages, LECL provides a structured way to write expressions and commands that the LangChain framework can interpret and execute.\n\n2. **Task Automation**: Users can script and automate repetitive tasks that involve language models, such as data preprocessing, text analysis, and report generation.\n\n3. **Integration with LangChain**: LECL is designed to work seamlessly within the La

While it provides a confident response, that response is entirely ficticious! Not a great look, OpenAI!

However, let's see what happens when we rework our prompts - and we add the content from the docs to our prompt as context.

In [48]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "No idea brah!".
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language or LCEL is a declarative way to easily compose chains together. It offers several benefits, including full sync, async, batch, and streaming support, easy handling of fallbacks, automatic parallelism for components that can be run simultaneously, and seamless integration with LangSmith for detailed tracing and debugging.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 276, 'total_tokens': 340}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None} id='run-50b24159-56dd-4f0f-83f3-f69ecc7e90d4-0' usage_metadata={'input_tokens': 276, 'output_tokens': 64, 'total_tokens': 340}


You'll notice that the response is much better this time. Not only does it answer the question well - but there's no trace of confabulation (hallucination) at all!

> NOTE: While RAG is an effective strategy to *help* ground LLMs, it is not nearly 100% effective. You will still need to ensure your responses are factual through some other processes

That, in essence, is the idea of RAG. We provide the model with context to answer our queries - and rely on it to translate the potentially lengthy and difficult to parse context into a natural language answer!

However, manually providing context is not scalable - and doesn't really offer any benefit.

Enter: Retrieval Pipelines.

## Task #4: Implement Naive RAG using LCEL

Now we can make a naive RAG application that will help us bridge the gap between our Pythonic implementation and a fully LangChain powered solution!

## Putting the R in RAG: Retrieval 101

In order to make our RAG system useful, we need a way to provide context that is most likely to answer our user's query to the LLM as additional context.

Let's tackle an immediate problem first: The Context Window.

All (most) LLMs have a limited context window which is typically measured in tokens. This window is an upper bound of how much stuff we can stuff in the model's input at a time.

Let's say we want to work off of a relatively large piece of source data - like the Ultimate Hitchhiker's Guide to the Galaxy. All 898 pages of it!

> NOTE: It is recommended you do not run the following cells, they are purely for demonstrative purposes.

In [21]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

We can leverage our tokenizer to count the number of tokens for us!

In [49]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

In [23]:
len(enc.encode(context))

12

The full set comes in at a whopping *636,144* tokens.

So, we have too much context. What can we do?

Well, the first thing that might enter your mind is: "Use a model with more context window", and we could definitely do that! However, even `gpt-4-128k` wouldn't be able to fit that whole text in the context window at once.

So, we can try splitting our document up into little pieces - that way, we can avoid providing too much context.

We have another problem now.

If we split our document up into little pieces, and we can't put all of them in the prompt. How do we decide which to include in the prompt?!

> NOTE: Content splitting/chunking strategies are an active area of research and iterative developement. There is no "one size fits all" approach to chunking/splitting at this moment. Use your best judgement to determine chunking strategies!

In order to conceptualize the following processes - let's create a toy context set!

### TextSplitting aka Chunking

We'll use the `RecursiveCharacterTextSplitter` to create our toy example.

It will split based on the following rules:

- Each chunk has a maximum size of 100 tokens
- It will try and split first on the `\n\n` character, then on the `\n`, then on the `<SPACE>` character, and finally it will split on individual tokens.

Let's implement it and see the results!

In [24]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [50]:
chunks = text_splitter.split_text(CONTEXT)

In [51]:
len(chunks)

3

In [52]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at ev

🏗️ Activity #1:
While there's nothing specifically wrong with the chunking method used above - it is a naive approach that is not sensitive to specific data formats.

Brainstorm some ideas that would split large single documents into smaller documents.

- `Semantic Chunking:using paragraph instead of lines, or sentences. Each paragraph can be treated as a coherent thought or idea, making it a natural unit for embedding. `
- ` Dynamic Contextual Chunking: Like we used before having chunk overlaps slightly with the previous one.Basically a sliding window approach where each ensuring that the context is preserved across chunks.`
- `Machine Learning: ML Algorithms that try solving the chunkins task in an effective manner`

As is shown in our result, we've split each section into 100 token chunks - cleanly separated by `\n\n` characters!

## Embeddings and Dense Vector Search

Now that we have our individual chunks, we need a system to correctly select the relevant pieces of information to answer our query.

This sounds like a perfect job for embeddings!

We'll be using OpenAI's `text-embedding-3` model as our embedding model today!

Let's load it up through LangChain.

In [28]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

####❓ Question #2:

What is the embedding dimension, given that we're using `text-embedding-3-small`?
```1536 is the embedding for the text-embedding-3-small. The embedding dimension is the length of the vector used to represent each item in the embedding space. so in the case of text-embedding-3-small each word or sentence will be represented by a vector with 1536 elements (or features)```

> HINT: Check out the [docs](https://platform.openai.com/docs/guides/embeddings) to help you answer this question.

### Finding the Embeddings for Our Chunks

First, let's find all our embeddings for each chunk and store them in a convenient format for later.

In [29]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [42]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"numerical vectors: {v}") 
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
numerical vector: [0.0005607009516097605, -0.0366472490131855, 0.023857517167925835, -0.008668888360261917, 0.050974905490875244, 0.03672611713409424, -0.006112256553024054, -0.01965123414993286, -0.023528901860117912, 0.011560709215700626, 0.021951545029878616, -0.02379179559648037, -0.0034833289682865143, -0.012507122941315174, -0.019927270710468292, -0.0064178695902228355, 0.022503620013594627, -0.024002108722925186, 0.01618104986846447, 0.025395439937710762, -0.004620340187102556, -0.013052625581622124, 0.02

Okay, great. Let's create a query - and then embed it!

In [31]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


Now, let's compare it against each existing chunk's embedding by using cosine similarity.

In [32]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [33]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.5372984870519116


And we get the expected result, which is the passage that specifically mentions prototyping in a Jupyter Notebook!

### Creating a Retriever

Now that we have an idea of how we're getting our most relevant information - let's see how we could create a pipeline that would automatically extract the closest chunk to our query and use it as context for our prompt!

First, we'll wrap the above in a helper function!

In [34]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

Now, let's add it to our pipeline!

In [35]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [36]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?',
 'response': AIMessage(content='Yes, LCEL can help take code from the notebook to production. It allows for easy composition of chains with full support for sync, async, batch, and streaming interfaces. This means you can prototype a chain in a Jupyter notebook using the sync interface and then seamlessly expose it as an async streaming interface for production.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 152, 'total_tokens': 216}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'stop', 'logprobs': None}, id='run-8dd622f6-6656-4c22-ad50-ba75d08873c1-0', usage_metadata={'input_tokens': 152, 'output_tokens': 64, 'total_tokens': 216}),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this mann

####❓ Question #3:

What does LCEL do that makes it more reliable at scale?
```The ability to do Chain Composition which provide the abiliy to create intricate worklofws that involve multiple steps and decision points. Additionally asynchronous execution and parallel processing can execugte multiple parts of a chain asynchronously allowing for a reliable processing at scale```

> HINT: Use your newly created `simple_rag` to help you answer this question!